In [1]:
import math
import pandas as pd 
import numpy as np

In [2]:
movies_df = pd.read_csv("generated_data/movies_small.csv")
movies_df = movies_df.set_index("movieId")
movies_df = movies_df[~movies_df.index.duplicated(keep="first")] # drop rows with duplicate indices

In [3]:
movies_df

,adult,belongs_to_collection,budget,genres,homepage,original_language,original_title,overview,popularity,poster_path,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
movieId,,,,,,,,,,,,,,,,,,,,,
1,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
2,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
3,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.712900,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
4,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
5,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161918,False,"{'id': 286023, 'name': 'Sharknado Collection',...",0,"[{'id': 35, 'name': 'Comedy'}, {'id': 27, 'nam...",http://www.syfy.com/sharknado4,en,Sharknado 4: The 4th Awakens,The new installment of the Sharknado franchise...,4.574494,/jcP3HFXF1BIW9LmBrDusbbDZjBG.jpg,...,2016-07-31,0.0,85.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"What happens in Vegas, stays in Vegas. Unless ...",Sharknado 4: The 4th Awakens,False,4.3,88.0
161944,False,NaN,8000000,"[{'id': 18, 'name': 'Drama'}]",NaN,en,The Last Brickmaker in America,A man must cope with the loss of his wife and ...,0.038998,/yWp7PgydSlxlhl7benKhTnCvRjN.jpg,...,2001-09-23,0.0,85.0,[],Released,NaN,The Last Brickmaker in America,False,7.0,1.0
162542,False,NaN,1000000,"[{'id': 53, 'name': 'Thriller'}, {'id': 10749,...",NaN,hi,रुस्तम,"Rustom Pavri, an honourable officer of the Ind...",7.333139,/q1lrN6ZrIsOs077lQB86aPGKZRF.jpg,...,2016-08-12,0.0,150.0,"[{'iso_639_1': 'hi', 'name': 'हिन्दी'}]",Released,Decorated Officer. Devoted Family Man. Defendi...,Rustom,False,7.3,25.0


In [4]:
ratings_df = pd.read_csv("data/ratings_small.csv")
ratings_df = ratings_df[ratings_df["movieId"].isin(movies_df.index)]

In [5]:
# predict ratings using average rating of previously seen movies
mean_ratings_df = ratings_df.groupby("userId").mean()[["rating"]].rename(columns={ "rating": "predicted_rating" })
ratings_df = ratings_df.join(mean_ratings_df, on="userId", validate="m:1")

In [6]:
ratings_df

,userId,movieId,rating,timestamp,predicted_rating
0,1,31,2.5,1260759144,2.550000
1,1,1029,3.0,1260759179,2.550000
2,1,1061,3.0,1260759182,2.550000
3,1,1129,2.0,1260759185,2.550000
4,1,1172,4.0,1260759205,2.550000
...,...,...,...,...,...
99999,671,6268,2.5,1065579370,3.917391
100000,671,6269,4.0,1065149201,3.917391
100001,671,6365,4.0,1070940363,3.917391
100002,671,6385,2.5,1070979663,3.917391


In [7]:
print("Baseline RMSE", np.sqrt(np.square(ratings_df["rating"] - ratings_df["predicted_rating"]).mean()))

Baseline RMSE 0.9552819408178532


In [8]:
total_expected_correct_recommendations = 0 
count = 0 

for user_id in ratings_df.userId.unique():
    assert(user_id in ratings_df["userId"].unique())
    
    df = ratings_df[ratings_df["userId"] == user_id]
    d = len(df) - int(len(df) * 0.80) # size of test
    
    temp = 3 * math.comb(3, 3) * math.comb(d - 3, 0) * math.factorial(d - 3) * math.factorial(3) + \
           2 * math.comb(3, 2) * math.comb(d - 3, 1) * math.factorial(d - 3) * math.factorial(3) + \
           1 * math.comb(3, 1) * math.comb(d - 3, 2) * math.factorial(d - 3) * math.factorial(3)
    
    total_expected_correct_recommendations += temp / math.factorial(d)
    count += 1

average_expected_correct_recommendations = total_expected_correct_recommendations / count 
print(f"Baseline correctly recommended {average_expected_correct_recommendations} of the top 3 movies on average")

Baseline correctly recommended 0.7865260392328405 of the top 3 movies on average
